# Produção de Petróleo, Líquidos de Gás Natural (LGN) e Gás Natural

**Autor:** Antonino Marques Jares

**Fonte:** Dados Abertos do GovBR

In [47]:
import pandas as pd
from pathlib import Path

# Dicionário de mapeamento (abreviação → número com dois dígitos)
mes_para_numero = {
    'JAN': '01', 'FEV': '02', 'MAR': '03', 'ABR': '04', 'MAI': '05', 'JUN': '06',
    'JUL': '07', 'AGO': '08', 'SET': '09', 'OUT': '10', 'NOV': '11', 'DEZ': '12'
}

petroleo = Path(rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\producao-petroleo-m3-1997-2025.csv")
df_petroleo = pd.read_csv(petroleo, sep=';', encoding='utf-8', low_memory=False)
df_petroleo['PAIS'] = 'BRASIL'
df_petroleo = df_petroleo.rename(columns={
    'MÊS': 'MES',
    'GRANDE REGIÃO': 'REGIAO',
    'UNIDADE DA FEDERAÇÃO': 'UF',
    'LOCALIZAÇÃO': 'LOCAL',
    'PRODUÇÃO': 'PRODUCAO'
})
df_petroleo['MES'] = df_petroleo['MES'].str.upper().map(mes_para_numero)
# Crie MES_ANO
df_petroleo['MES_ANO'] = df_petroleo['MES'] + '/' + df_petroleo['ANO'].astype(str)
df_petroleo = df_petroleo[['PAIS','MES_ANO','PRODUTO','PRODUCAO']]

gas_natural = Path(rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\producao-gas-natural-1000m3-1997-2025.csv")
df_gas_natural = pd.read_csv(gas_natural, sep=';', encoding='utf-8', low_memory=False)
df_gas_natural['PAIS'] = 'BRASIL'
df_gas_natural = df_gas_natural.rename(columns={
    'MÊS': 'MES',
    'GRANDE REGIÃO': 'REGIAO',
    'UNIDADE DA FEDERAÇÃO': 'UF',
    'LOCALIZAÇÃO': 'LOCAL',
    'PRODUÇÃO': 'PRODUCAO'
})
df_gas_natural['MES'] = df_gas_natural['MES'].str.upper().map(mes_para_numero)
# Crie MES_ANO
df_gas_natural['MES_ANO'] = df_gas_natural['MES'] + '/' + df_gas_natural['ANO'].astype(str)
df_gas_natural['PRODUCAO'] = df_gas_natural['PRODUCAO'] 
df_gas_natural = df_gas_natural[['PAIS','MES_ANO','PRODUTO','PRODUCAO']]


lgn = Path(rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\producao-lgn-m3-1997-2025.csv")
df_lgn = pd.read_csv(lgn, sep=';', encoding='utf-8', low_memory=False)
df_lgn['PAIS'] = 'BRASIL'
df_lgn = df_lgn.rename(columns={
    'MÊS': 'MES',
    'GRANDE REGIÃO': 'REGIAO',
    'UNIDADE DA FEDERAÇÃO': 'UF',
    'LOCALIZAÇÃO': 'LOCAL',
    'PRODUÇÃO': 'PRODUCAO'
})
df_lgn['MES'] = df_lgn['MES'].str.upper().map(mes_para_numero)
# Crie MES_ANO
df_lgn['MES_ANO'] = df_lgn['MES'] + '/' + df_lgn['ANO'].astype(str)
df_lgn = df_lgn[['PAIS','MES_ANO','PRODUTO','PRODUCAO']]

print(df_petroleo.head())
print(df_gas_natural.head())
print(df_lgn.head())




     PAIS  MES_ANO   PRODUTO PRODUCAO
0  BRASIL  01/1997  PETRÓLEO   122200
1  BRASIL  02/1997  PETRÓLEO   124785
2  BRASIL  03/1997  PETRÓLEO   128177
3  BRASIL  04/1997  PETRÓLEO   124968
4  BRASIL  05/1997  PETRÓLEO   115710
     PAIS  MES_ANO      PRODUTO PRODUCAO
0  BRASIL  01/1997  GÁS NATURAL    50891
1  BRASIL  02/1997  GÁS NATURAL    48632
2  BRASIL  03/1997  GÁS NATURAL    50528
3  BRASIL  04/1997  GÁS NATURAL    50588
4  BRASIL  05/1997  GÁS NATURAL    44179
     PAIS  MES_ANO PRODUTO PRODUCAO
0  BRASIL  12/1997     LGN     1104
1  BRASIL  04/1997     LGN    50970
2  BRASIL  03/1997     LGN    50206
3  BRASIL  02/1997     LGN    44709
4  BRASIL  01/1997     LGN    48812


# Criando DB a partir da união dos DATA FRAMES petróleo e lgn

In [48]:
import pandas as pd
import sqlite3

# UNIDADE DE MEDIDA METROS CUBICOS
df = pd.concat(
    [df_petroleo, df_lgn],
    axis=0,
    ignore_index=True
)



try:
    conn = sqlite3.connect('producao_petroleo_lgn.db')
    # Salvar na tabela 'producao_ptl_lgn' do SQLite
    df.to_sql(
        name='producao',          # Nome da tabela
        con=conn,                 # Conexão com o banco
        if_exists='replace',      # Substitui a tabela se existir ('append' para adicionar)
        index=False,              # Não incluir o índice
        dtype={                   # Especificar tipos de dados (opcional)
            'PAIS': 'TEXT',
            'MES_ANO': 'TEXT',
            'PRODUTO': 'TEXT',
            'PRODUCAO': 'NUMERIC'
        }
    )

    
finally:
    # Fechar a conexão
    if conn:
        conn.close()


# Criando DB de Gás Natural

In [49]:
import pandas as pd
import sqlite3

try:
    conn = sqlite3.connect('producao_gas_natural.db')
    # Salvar na tabela 'producao_ptl_lgn' do SQLite
    df_gas_natural.to_sql(
        name='producao',          # Nome da tabela
        con=conn,                 # Conexão com o banco
        if_exists='replace',      # Substitui a tabela se existir ('append' para adicionar)
        index=False,              # Não incluir o índice
        dtype={                   # Especificar tipos de dados (opcional)
            'PAIS': 'TEXT',
            'MES_ANO': 'TEXT',
            'PRODUTO': 'TEXT',
            'PRODUCAO': 'NUMERIC'
        }
    )

    
finally:
    # Fechar a conexão
    if conn:
        conn.close()


# Agrupando a produção (petróleo e lgn) por PAIS, MES_ANO e PRODUTO

In [50]:
import sqlite3
import pandas as pd
from pathlib import Path

# Processar ambos os bancos de dados
try:
    # 1. DB1 (Petróleo e LGN)
    conn = sqlite3.connect('producao_petroleo_lgn.db')
    cursor = conn.cursor()
    
    # Verificar tabelas existentes
    df = pd.read_sql("""SELECT PAIS, MES_ANO, PRODUTO, SUM(PRODUCAO) as PRODUCAO
                        FROM producao
                        WHERE PRODUCAO > 0
                        GROUP BY PAIS, MES_ANO, PRODUTO
                        ORDER BY MES_ANO""", conn)
    # Salvar o CSV
    resumo_path = rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\resumo_producao_petroleo_lgn.csv"
    df.to_csv(resumo_path, index=False, encoding='utf-8-sig')

except Exception as e:
    print(f"Erro geral: {str(e)}")
finally:
    # Fechar a conexão
    if conn:
        conn.close()


# Agrupando a produção (gás natural) por PAIS, MES_ANO e PRODUTO

In [51]:
import sqlite3
import pandas as pd
from pathlib import Path

try:
    conn = sqlite3.connect('producao_gas_natural.db')
    cursor = conn.cursor()
    
    # Verificar tabelas existentes
    df = pd.read_sql("""SELECT PAIS, MES_ANO, PRODUTO, SUM(PRODUCAO) as PRODUCAO
                        FROM producao
                        WHERE PRODUCAO > 0
                        GROUP BY PAIS, MES_ANO, PRODUTO
                        ORDER BY MES_ANO""", conn)
    # Salvar o CSV
    resumo_path = rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\resumo_producao_gas_natural.csv"
    df.to_csv(resumo_path, index=False, encoding='utf-8-sig')

except Exception as e:
    print(f"Erro geral: {str(e)}")
finally:
    # Fechar a conexão
    if conn:
        conn.close()


# Gerando HTML com o gráfico de Produção - Petróleo e LGN

In [52]:
import pandas as pd

# Ler o arquivo CSV
resumo_path = rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\resumo_producao_petroleo_lgn.csv"
df = pd.read_csv(resumo_path)

# Converter MES_ANO para datetime (assumindo formato MM/AAAA)
df['DATA'] = pd.to_datetime(df['MES_ANO'], format='%m/%Y')

# Ordenar por data
df = df.sort_values('DATA')

# Criar o arquivo HTML
html_content = f"""
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Histórico de produção mensal por metro cúbico no Brasil</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
        }}
        .chart-container {{
            width: 80%;
            margin: 0 auto;
        }}
        h1 {{
            text-align: center;
            color: #333;
        }}
    </style>
</head>
<body>
    <h1>Histórico de produção mensal por metro cúbico no Brasil</h1>
    <div class="chart-container">
        <canvas id="productionChart"></canvas>
    </div>

    <script>
        // Dados do gráfico
        const productionData = {{
            labels: {df['DATA'].dt.strftime('%m/%Y').unique().tolist()},
            datasets: [
                {{
                    label: 'Petróleo',
                    data: {df[df['PRODUTO'] == 'PETRÓLEO'].sort_values('DATA')['PRODUCAO'].tolist()},
                    borderColor: 'rgb(75, 192, 192)',
                    backgroundColor: 'rgba(75, 192, 192, 0.2)',
                    tension: 0.1
                }},
                {{
                    label: 'LGN',
                    data: {df[df['PRODUTO'] == 'LGN'].sort_values('DATA')['PRODUCAO'].tolist()},
                    borderColor: 'rgb(54, 162, 235)',
                    backgroundColor: 'rgba(54, 162, 235, 0.2)',
                    tension: 0.1
                }}
            ]
        }};

        // Configuração do gráfico
        const config = {{
            type: 'line',
            data: productionData,
            options: {{
                responsive: true,
                plugins: {{
                    title: {{
                        display: true,
                        text: 'Produtos'
                    }},
                }},
                scales: {{
                    y: {{
                        beginAtZero: false,
                        title: {{
                            display: true,
                            text: 'Metros Cúbicos'
                        }}
                    }},
                    x: {{
                        title: {{
                            display: true,
                            text: 'Mês/Ano'
                        }}
                    }}
                }}
            }},
        }};

        // Renderizar o gráfico
        window.onload = function() {{
            const ctx = document.getElementById('productionChart').getContext('2d');
            new Chart(ctx, config);
        }};
    </script>
</body>
</html>
"""

# Salvar o arquivo HTML
html_path = rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\historico_producao_petroleo_e_lgn.html"
with open(html_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"Arquivo HTML gerado com sucesso: {html_path}")

Arquivo HTML gerado com sucesso: C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\historico_producao_petroleo_e_lgn.html


# Gerando HTML com o gráfico de Produção - Gás Natural

In [53]:
import pandas as pd

# Ler o arquivo CSV
resumo_path = rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\resumo_producao_gas_natural.csv"
df = pd.read_csv(resumo_path)

# Converter MES_ANO para datetime (assumindo formato MM/AAAA)
df['DATA'] = pd.to_datetime(df['MES_ANO'], format='%m/%Y')

# Ordenar por data
df = df.sort_values('DATA')

# Criar o arquivo HTML
html_content = f"""
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Histórico de produção mensal por 1000 metros cúbicos no Brasil</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
        }}
        .chart-container {{
            width: 80%;
            margin: 0 auto;
        }}
        h1 {{
            text-align: center;
            color: #333;
        }}
    </style>
</head>
<body>
    <h1>Histórico de produção mensal por 1000 metros cúbicos no Brasil</h1>
    <div class="chart-container">
        <canvas id="productionChart"></canvas>
    </div>

    <script>
        // Dados do gráfico
        const productionData = {{
            labels: {df['DATA'].dt.strftime('%m/%Y').unique().tolist()},
            datasets: [
                {{
                    label: 'GÁS NATURAL',
                    data: {df[df['PRODUTO'] == 'GÁS NATURAL'].sort_values('DATA')['PRODUCAO'].tolist()},
                    borderColor: 'rgb(75, 192, 192)',
                    backgroundColor: 'rgba(75, 192, 192, 0.2)',
                    tension: 0.1
                }}
            ]
        }};

        // Configuração do gráfico
        const config = {{
            type: 'line',
            data: productionData,
            options: {{
                responsive: true,
                plugins: {{
                    title: {{
                        display: true,
                        text: 'Produtos'
                    }},
                }},
                scales: {{
                    y: {{
                        beginAtZero: false,
                        title: {{
                            display: true,
                            text: '1000 Metros Cúbicos'
                        }}
                    }},
                    x: {{
                        title: {{
                            display: true,
                            text: 'Mês/Ano'
                        }}
                    }}
                }}
            }},
        }};

        // Renderizar o gráfico
        window.onload = function() {{
            const ctx = document.getElementById('productionChart').getContext('2d');
            new Chart(ctx, config);
        }};
    </script>
</body>
</html>
"""

# Salvar o arquivo HTML
html_path = rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\historico_producao_gas_natural.html"
with open(html_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"Arquivo HTML gerado com sucesso: {html_path}")

Arquivo HTML gerado com sucesso: C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\historico_producao_gas_natural.html


In [ ]:
import pandas as pd

# Ler os arquivos CSV
df_petroleo_lgn = pd.read_csv(rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\resumo_producao_petroleo_lgn.csv")
df_gas_natural = pd.read_csv(rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\resumo_producao_gas_natural.csv")

# Processar dados (petróleo/LGN)
df_petroleo_lgn['DATA'] = pd.to_datetime(df_petroleo_lgn['MES_ANO'], format='%m/%Y')
df_petroleo_lgn_sorted = df_petroleo_lgn.sort_values('DATA')

# Processar dados (gás natural)
df_gas_natural['DATA'] = pd.to_datetime(df_gas_natural['MES_ANO'], format='%m/%Y')
df_gas_natural_sorted = df_gas_natural.sort_values('DATA')

# Criar o HTML unificado
html_content = f"""
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Produção de Petróleo, LGN e Gás Natural no Brasil</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/chartjs-plugin-zoom@2.0.0"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: white;
        }}
        .header {{
            text-align: center;
            color: #333;
            margin-bottom: 30px;
        }}
        .chart-wrapper {{
            width: 95%;
            max-width: 1200px;
            margin: 0 auto 50px auto;
            background-color: white;
            padding: 20px;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
            border-radius: 8px;
        }}
        .chart-container {{
            height: 500px;
            margin-top: 10px;
        }}
        .chart-title {{
            text-align: center;
            margin-bottom: 10px;
            color: #2c3e50;
        }}
        .chart-controls {{
            display: flex;
            justify-content: center;
            gap: 10px;
            margin-bottom: 15px;
        }}
        .chart-btn {{
            padding: 8px 15px;
            background-color: #f0f0f0;
            border: 1px solid #ddd;
            border-radius: 4px;
            cursor: pointer;
            font-size: 14px;
            transition: all 0.3s;
        }}
        .chart-btn:hover {{
            background-color: #e0e0e0;
        }}
        canvas {{
            width: 100% !important;
            height: 100% !important;
        }}
    </style>
</head>
<body>
    <div class="header">
        <h1>Produção Mensal de Petróleo, LGN e Gás Natural no Brasil</h1>
    </div>
    
    <!-- Gráfico 1: Petróleo e LGN -->
    <div class="chart-wrapper">
        <h2 class="chart-title">Produção de Petróleo e LGN (m³)</h2>
        
        <div class="chart-controls">
            <button class="chart-btn" onclick="zoomIn('petroleoLgnChart')">Zoom In (+)</button>
            <button class="chart-btn" onclick="zoomOut('petroleoLgnChart')">Zoom Out (-)</button>
            <button class="chart-btn" onclick="panLeft('petroleoLgnChart')">← Esquerda</button>
            <button class="chart-btn" onclick="panRight('petroleoLgnChart')">Direita →</button>
            <button class="chart-btn" onclick="resetZoom('petroleoLgnChart')">Resetar Zoom</button>
        </div>
        
        <div class="chart-container">
            <canvas id="petroleoLgnChart"></canvas>
        </div>
    </div>
    
    <!-- Gráfico 2: Gás Natural -->
    <div class="chart-wrapper">
        <h2 class="chart-title">Produção de Gás Natural (mil m³)</h2>
        
        <div class="chart-controls">
            <button class="chart-btn" onclick="zoomIn('gasNaturalChart')">Zoom In (+)</button>
            <button class="chart-btn" onclick="zoomOut('gasNaturalChart')">Zoom Out (-)</button>
            <button class="chart-btn" onclick="panLeft('gasNaturalChart')">← Esquerda</button>
            <button class="chart-btn" onclick="panRight('gasNaturalChart')">Direita →</button>
            <button class="chart-btn" onclick="resetZoom('gasNaturalChart')">Resetar Zoom</button>
        </div>
        
        <div class="chart-container">
            <canvas id="gasNaturalChart"></canvas>
        </div>
    </div>

    <script>
        // Variáveis globais para armazenar os charts
        let petroleoLgnChart, gasNaturalChart;
        
        // Funções de controle para os gráficos
        function zoomIn(chartId) {{
            const chart = getChartInstance(chartId);
            if (chart) {{
                chart.zoom(1.1);
            }}
        }}
        
        function zoomOut(chartId) {{
            const chart = getChartInstance(chartId);
            if (chart) {{
                chart.zoom(0.9);
            }}
        }}
        
        function panLeft(chartId) {{
            const chart = getChartInstance(chartId);
            if (chart) {{
                chart.pan({{x: -100}}, undefined, 'default');
            }}
        }}
        
        function panRight(chartId) {{
            const chart = getChartInstance(chartId);
            if (chart) {{
                chart.pan({{x: 100}}, undefined, 'default');
            }}
        }}
        
        function resetZoom(chartId) {{
            const chart = getChartInstance(chartId);
            if (chart) {{
                chart.resetZoom();
            }}
        }}
        
        function getChartInstance(chartId) {{
            switch(chartId) {{
                case 'petroleoLgnChart': return petroleoLgnChart;
                case 'gasNaturalChart': return gasNaturalChart;
                default: return null;
            }}
        }}

        // Dados do gráfico de Petróleo e LGN
        const petroleoLgnData = {{
            labels: {df_petroleo_lgn_sorted['DATA'].dt.strftime('%m/%Y').unique().tolist()},
            datasets: [
                {{
                    label: 'Petróleo',
                    data: {df_petroleo_lgn_sorted[df_petroleo_lgn_sorted['PRODUTO'] == 'PETRÓLEO']['PRODUCAO'].tolist()},
                    borderColor: 'rgb(75, 192, 192)',
                    backgroundColor: 'rgba(75, 192, 192, 0.2)',
                    borderWidth: 2,
                    tension: 0.1,
                    fill: true
                }},
                {{
                    label: 'LGN',
                    data: {df_petroleo_lgn_sorted[df_petroleo_lgn_sorted['PRODUTO'] == 'LGN']['PRODUCAO'].tolist()},
                    borderColor: 'rgb(54, 162, 235)',
                    backgroundColor: 'rgba(54, 162, 235, 0.2)',
                    borderWidth: 2,
                    tension: 0.1,
                    fill: true
                }}
            ]
        }};

        // Configuração do gráfico de Petróleo e LGN
        const petroleoLgnConfig = {{
            type: 'line',
            data: petroleoLgnData,
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                plugins: {{
                    legend: {{
                        position: 'top',
                    }},
                    tooltip: {{
                        mode: 'index',
                        intersect: false
                    }},
                    zoom: {{
                        zoom: {{
                            wheel: {{
                                enabled: true,
                            }},
                            pinch: {{
                                enabled: true
                            }},
                            mode: 'xy',
                        }},
                        pan: {{
                            enabled: true,
                            mode: 'xy',
                        }},
                        limits: {{
                            x: {{min: 'original', max: 'original'}},
                            y: {{min: 'original', max: 'original'}}
                        }}
                    }}
                }},
                scales: {{
                    y: {{
                        beginAtZero: false,
                        title: {{ 
                            display: true, 
                            text: 'Metros Cúbicos (m³)',
                            font: {{
                                weight: 'bold'
                            }}
                        }},
                        grid: {{
                            color: 'rgba(0, 0, 0, 0.05)'
                        }}
                    }},
                    x: {{
                        title: {{ 
                            display: true, 
                            text: 'Mês/Ano',
                            font: {{
                                weight: 'bold'
                            }}
                        }},
                        grid: {{
                            display: false
                        }}
                    }}
                }}
            }}
        }};

        // Dados do gráfico de Gás Natural
        const gasNaturalData = {{
            labels: {df_gas_natural_sorted['DATA'].dt.strftime('%m/%Y').unique().tolist()},
            datasets: [
                {{
                    label: 'Gás Natural',
                    data: {df_gas_natural_sorted[df_gas_natural_sorted['PRODUTO'] == 'GÁS NATURAL']['PRODUCAO'].tolist()},
                    borderColor: 'rgb(255, 99, 132)',
                    backgroundColor: 'rgba(255, 99, 132, 0.2)',
                    borderWidth: 2,
                    tension: 0.1,
                    fill: true
                }}
            ]
        }};

        // Configuração do gráfico de Gás Natural
        const gasNaturalConfig = {{
            type: 'line',
            data: gasNaturalData,
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                plugins: {{
                    legend: {{
                        position: 'top',
                    }},
                    tooltip: {{
                        mode: 'index',
                        intersect: false
                    }},
                    zoom: {{
                        zoom: {{
                            wheel: {{
                                enabled: true,
                            }},
                            pinch: {{
                                enabled: true
                            }},
                            mode: 'xy',
                        }},
                        pan: {{
                            enabled: true,
                            mode: 'xy',
                        }},
                        limits: {{
                            x: {{min: 'original', max: 'original'}},
                            y: {{min: 'original', max: 'original'}}
                        }}
                    }}
                }},
                scales: {{
                    y: {{
                        beginAtZero: false,
                        title: {{ 
                            display: true, 
                            text: 'Mil Metros Cúbicos (1000 m³)',
                            font: {{
                                weight: 'bold'
                            }}
                        }},
                        grid: {{
                            color: 'rgba(0, 0, 0, 0.05)'
                        }}
                    }},
                    x: {{
                        title: {{ 
                            display: true, 
                            text: 'Mês/Ano',
                            font: {{
                                weight: 'bold'
                            }}
                        }},
                        grid: {{
                            display: false
                        }}
                    }}
                }}
            }}
        }};

        // Renderizar os gráficos
        document.addEventListener('DOMContentLoaded', function() {{
            const ctx1 = document.getElementById('petroleoLgnChart');
            petroleoLgnChart = new Chart(ctx1, petroleoLgnConfig);
            
            const ctx2 = document.getElementById('gasNaturalChart');
            gasNaturalChart = new Chart(ctx2, gasNaturalConfig);
        }});
    </script>
</body>
</html>
"""

# Salvar o arquivo HTML unificado
html_path = rf"C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\historico_producao_unificado.html"
with open(html_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"Arquivo HTML unificado gerado com sucesso: {html_path}")

Arquivo HTML unificado gerado com sucesso: C:\Users\Nino\AnacondaProjects\producao_petroleo_lgn_gas_natural\historico_producao_unificado.html
